In [0]:
%sql
create volume if not exists dev.demo_db.files

In [0]:
raw_df = (spark.read
          .format("csv")
          .option("header","true")
          .option("inferSchema","true")
          .load("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"))

raw_df.display()

In [0]:
from pyspark.sql.functions import to_date,to_timestamp,round,year


In [0]:
staging_df = (raw_df.withColumnRenamed("Call Number","CallNumber")
                .withColumnRenamed("Unit ID","UnitID")
                .withColumnRenamed ("Incident Number","IncidentNumber")
                .withColumnRenamed("Call Date","CallDate")
                .withColumnRenamed("Watch Date","WatchDate")
                .withColumnRenamed("Call Final Disposition","CallFinalDisposition")
                .withColumnRenamed("Available DtTm","AvailableDtTm")
                .withColumnRenamed("Zipcode of Incident","Zipcode")
                .withColumnRenamed("Station Area","StationArea")
                .withColumnRenamed("Final Priority","FinalPriority")
                .withColumnRenamed("ALS Unit","ALSUnit")
                .withColumnRenamed("Call Type Group","CallTypeGroup")
                .withColumnRenamed("Unit sequence in call dispatch","UnitSequenceInCallDispatch")
                .withColumnRenamed("Fire Prevention District","FirePreventionDistrict")
                .withColumnRenamed("Supervisor District","SupervisorDistrict")
                .withColumn("CallDate",to_date("CallDate","MM/dd/yyyy"))
                .withColumn("WatchDate",to_date("WatchDate","MM/dd/yyyy"))
                .withColumn("AvailableDtTm",to_timestamp("AvailableDtTm","MM/dd/yyyy hh:mm:ss a"))
                .withColumn("Delay",round("Delay",2))
                .withColumn("Year",year("CallDate")))

staging_df.display()

In [0]:
(staging_df.write
 .format("delta")
 .mode("overwrite")
 .save("/Volumes/dev/demo_db/files/fire_calls_tbl")
 )

In [0]:
(staging_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("dev.demo_db.fire_calls_tbl")
 )

In [0]:
%fs ls /Volumes/dev/demo_db/files/fire_calls_tbl/

In [0]:
%fs ls /Volumes/dev/demo_db/files/fire_calls_tbl/_delta_log/

In [0]:
%sql
delete from delta.`/Volumes/dev/demo_db/files/fire_calls_tbl/`
where CallFinalDisposition = "Duplicate"

In [0]:
%fs ls /Volumes/dev/demo_db/files/fire_calls_tbl/_delta_log/

In [0]:
%fs ls /Volumes/dev/demo_db/files/fire_calls_tbl

In [0]:
%sql
describe history delta.`/Volumes/dev/demo_db/files/fire_calls_tbl`

In [0]:
%sql
select * from json.`/Volumes/dev/demo_db/files/fire_calls_tbl/_delta_log/00000000000000000001.json`